In [1]:
import numpy as np
import cv2

image = cv2.imread("sar_2.jpg", cv2.IMREAD_GRAYSCALE)
image = image.astype(np.float32)
image = image[:image.shape[0] - image.shape[0] % 2, :image.shape[1] - image.shape[1] % 2]
np.savetxt('original_image.txt', image, fmt='%.0f')

In [2]:
def haar_wavelet_transform(image):
    h, w = image.shape
    half_h, half_w = h // 2, w // 2
    transformed = np.zeros_like(image, dtype=np.float32)

    for i in range(h):
        row = image[i, :]
        avg = (row[0::2] + row[1::2]) * 0.5
        diff = (row[0::2] - row[1::2]) * 0.5
        transformed[i, :half_w] = avg
        transformed[i, half_w:] = diff

    result = np.zeros_like(transformed, dtype=np.float32)
    for j in range(w):
        col = transformed[:, j]
        avg = (col[0::2] + col[1::2]) * 0.5
        diff = (col[0::2] - col[1::2]) * 0.5
        result[:half_h, j] = avg
        result[half_h:, j] = diff

    LL = result[:half_h, :half_w]
    LH = result[:half_h, half_w:]
    HL = result[half_h:, :half_w]
    HH = result[half_h:, half_w:]

    return LL, LH, HL, HH


LL, LH, HL, HH = haar_wavelet_transform(image)

In [3]:
def quantize_high_freq(data, levels=4):
    dmin, dmax = np.min(data), np.max(data)
    if dmax == dmin:
        return np.zeros_like(data, dtype=int), dmin, 1

    step = (dmax - dmin) / (levels - 1)
    quantized = np.round((data - dmin) / step).astype(int)
    quantized = np.clip(quantized, 0, levels - 1)
    return quantized, dmin, step

LH_q, _, _ = quantize_high_freq(LH, 4)
HL_q, _, _ = quantize_high_freq(HL, 4)
HH_q, _, _ = quantize_high_freq(HH, 4)

In [4]:
def run_length_encode(array):
    flat = array.flatten()
    encoded = []
    prev = flat[0]
    count = 1
    for value in flat[1:]:
        if value == prev:
            count += 1
        else:
            encoded.append((prev, count))
            prev = value
            count = 1
    encoded.append((prev, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

In [5]:
with open('wavelet_data.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%.2f')
    for v, c in LH_rle:
        f.write(f"{v} {c}\n")
    for v, c in HL_rle:
        f.write(f"{v} {c}\n")
    for v, c in HH_rle:
        f.write(f"{v} {c}\n")

In [6]:
original_size = image.nbytes

with open('wavelet_data.txt', 'r') as f:
    compressed_size = len(f.read().encode('utf-8'))

size_diff = original_size - compressed_size

percent = (size_diff / original_size) * 100

print(f"Размер исходного изображения: {original_size} байт")
print(f"Размер после вейвлет-преобразования: {compressed_size} байт")
print(f"Экономия: {size_diff} байт")
print(f"Снижение объема данных: {percent:.2f}%")

Размер исходного изображения: 5750400 байт
Размер после вейвлет-преобразования: 2736342 байт
Экономия: 3014058 байт
Снижение объема данных: 52.41%
